In [ ]:
from torchvision import *
import torchvision as vision
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# model = torch.nn.Sequential(*(list(model.children())[:-1]))
df = pd.read_csv('../datasets/processed/pokemon.csv')
ohe = OneHotEncoder()
ohe.fit(df[['type1']])
types = ohe.transform(df[['type1']]).toarray()
chosen = [20, 100, 1, 5, 2]
df = df.query('dex  in @chosen')

In [ ]:
# create a new model using custom encoder and decoder
class VIencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = models.vit_b_16(weights='IMAGENET1K_V1')
        self.decoder = torch.nn.Sequential(
            nn.Linear(1018, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 150528),
            nn.Sigmoid(),
            )


    def forward(self, image, ptype):
        img = Image.open(image)
        img = img.convert('RGB')
        img = vision.transforms.Resize((224, 224))(img)
        img = vision.transforms.ToTensor()(img)
        img = img.unsqueeze(0)
        tensor = self.encoder(img)
        tensor = torch.cat((tensor[0,:], ptype))
        outputs = self.decoder(tensor)
        return outputs

In [ ]:
# # create a new model using custom encoder and decoder
# class VIencoder2(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.encoder = models.vit_b_16(weights='IMAGENET1K_V1')
#         self.decoder = torch.nn.Sequential(
#             nn.Conv2d(3, 64, 3, 1),
#             nn.ReLU(),
#             nn.Conv2d(64, 64, 3, 1),
#             nn.ReLU(),
#             nn.Conv2d(64, 64, 3, 1),
#             nn.ReLU(),
#             nn.Conv2d(64, 3, 3, 1),
#             nn.Sigmoid()
#             )


#     def forward(self, image):
#         img = Image.open(image)
#         img = img.convert('RGB')
#         img = vision.transforms.Resize((224, 224))(img)
#         img = vision.transforms.ToTensor()(img)
#         img = img.unsqueeze(0)
#         tensor = self.encoder(img)
#         print(tensor.shape)
#         tensor = tensor.resize(31, 32)
#         outputs = self.decoder(tensor)
#         return outputs

In [ ]:
# vie = VIencoder2()
vie = torch.load('pokemon_autoencoder.pt')

In [ ]:
# finetune model on pokemon dataset
epochs = 5000
learning_rate = 0.00001
optimizer = torch.optim.Adam(vie.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()

In [ ]:
for epoch in range(500):
    losses = []
    for i in df.index:
        optimizer.zero_grad()
        image = df.loc[i, 'image']
        img = Image.open(df['image'][i])
        ptype = torch.tensor(types[i, :]).float()
        img = img.convert('RGB')
        img = vision.transforms.Resize((224, 224))(img)
        img = vision.transforms.ToTensor()(img)
        img = img.unsqueeze(0)
        img = img.flatten()
        output = vie(image, ptype)
        loss = criterion(output, img)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    print(f'Epoch {epoch} Loss: {np.mean(losses)}')

In [ ]:
# save model
torch.save(vie, 'pokemon_autoencoder.pt')

In [ ]:
# df = pd.read_csv('../datasets/processed/pokemon.csv')
outputs = []
# show 5 reconstructed images
for i in df.index:
    # image = Image.open(df['image'][i])
    img = df['image'][i]
    ptype = torch.tensor(types[i, :]).float()

    output = vie(img, ptype)
    output = output.view(3, 224, 224)
    output = vision.transforms.ToPILImage()(output)
    outputs.append(output)
    plt.figure(i+10)
    plt.imshow(Image.open(img))
    plt.show()
    plt.figure(i+20)
    plt.imshow(output)
    plt.show()